# Analyzing CIA Factbook Data Using SQL
We are exploring and analyzing the data from the CIA Factbook to learn the fundamental skills and be familiar with SQL and its uses.

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` - the name of the country.
* `area` - the country's total area (both land and water).
* `area_land` - the country's land area in square kilometers.
* `area_water` - the country's waterarea in square kilometers.
* `population` - the country's population.
* `population_growth` - the country's population growth as a percentage.
* `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` - the country's death rate, or the number of death per year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

## Summary Statistics

In [5]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


## Outlier Countries
We see a few interesting things in the summary statistics on the previous screen:

* There's a country with a population of `0`
* There's a country with a population of `7256490011` (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [14]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population) FROM facts) OR population == (SELECT MAX(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The table contains a row for `Antarctica` and the whole `World`, which explains the populations `0` and `7.2 billion`. We should recalculate the summary statistics excluding the row for the whole world.

## Recalculating Summary Statistics

In [9]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE name != 'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


We can see that without the row for the `World`, there is a country who has the maximum population of 1.4 billion

## Finding the Average Population and Average Area

In [11]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
  FROM facts
 WHERE name != 'World';

Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


The average population of a country is 32 million with an average area of 555 thousand square kilometres

## Most Densely Populated Countries

In [17]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population > (SELECT AVG(population) 
                       FROM facts 
                      WHERE name != 'World'
                    ) AND 
       area < (SELECT AVG(area) 
                 FROM facts 
                WHERE name != 'World'
              );

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


There are 14 countries where their populations are above the average while having the area that is below average.

# Which countries will add the most people to their population next year?

In [18]:
%%sql
SELECT name, population, population_growth, (population * population_growth) AS pop_increase
  FROM facts
 WHERE name != 'World'
ORDER BY pop_increase DESC
 LIMIT 5;

Done.


name,population,population_growth,pop_increase
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.20000005
Pakistan,199085847,1.46,290665336.62
Ethiopia,99465819,2.89,287456216.91


The top 5 countries who will add the most people to their population next year are India, China, Nigeria, Pakistan, and Ethiopia. 

It is surprising that India will increase their population more than China, Nigeria, and Pakistan COMBINED.